# Community contribution

Implementing simple ChatGPT interface to maintain conversation and context with sleected model

In [0]:
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import Markdown, display, update_display, clear_output
from openai import OpenAI

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'


In [0]:
class ChatGPTInterface:
    def __init__(self, api_key, model, system_message="You are a helpful assistant. You can format your responses using Markdown."):
        self.openai = OpenAI(api_key=api_key)
        self.model = model
        self.conversation_history = [{"role": "system", "content": system_message}]

        self.chat_area = widgets.Output()
        self.input_box = widgets.Text(placeholder="Enter your message here...")
        self.send_button = widgets.Button(description="Send")
        self.clear_button = widgets.Button(description="Clear")

        self.send_button.on_click(self.send_message)
        self.clear_button.on_click(self.clear_chat)

        self.layout = widgets.VBox([
            self.chat_area,
            widgets.HBox([self.input_box, self.send_button, self.clear_button])
        ])

    def display(self):
        display(self.layout)

    def send_message(self, _):
        user_message = self.input_box.value.strip()
        if user_message:
            self.conversation_history.append({"role": "user", "content": user_message})
            self.display_message("You", user_message)
            self.input_box.value = ""

            try:
                response = self.openai.chat.completions.create(
                    model=self.model,
                    messages=self.conversation_history
                )
                assistant_message = response.choices[0].message.content.strip()
                self.conversation_history.append({"role": "assistant", "content": assistant_message})
                self.display_message("ChatGPT", assistant_message)
            except Exception as e:
                self.display_message("Error", str(e))

    def clear_chat(self, _):
        self.conversation_history = [{"role": "system", "content": self.conversation_history[0]["content"]}]
        self.chat_area.clear_output(wait=True)

    def display_message(self, sender, message):
        self.chat_area.append_display_data(Markdown(f"**{sender}:**\n{message}"))


In [0]:
chat_interface = ChatGPTInterface(api_key,MODEL)
chat_interface.display()